# 3. Generating Multiple Samples using MS1 Controller

In this notebook, we demonstrate how ViMMS can be used to generate multiple samples (sets of chemicals) that are biological and technical replicates. The MS1 controller is then used to produce mass spectral data in form of .mzML files for the multiple samples.

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
from collections import defaultdict
import os

In [4]:
import sys
sys.path.append('..')

In [5]:
from vimms.Chemicals import ChemicalCreator, MultiSampleCreator
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Controller import SimpleMs1Controller
from vimms.Common import *

Load previously trained KDEs in `PeakSampler` and the list of extracted metabolites, created in **01. Download Data.ipynb**.

In [6]:
base_dir = os.path.abspath('example_data')
ps = load_obj(os.path.join(base_dir, 'peak_sampler_mz_rt_int_19_beers_fullscan.p'))
hmdb = load_obj(os.path.join(base_dir, 'hmdb_compounds.p'))

Set ViMMS logging level

In [7]:
set_log_level_warning()
# set_log_level_info()
# set_log_level_debug()

## Create Initial Chemical

Define an output folder containing our results

In [8]:
out_dir = os.path.join(base_dir, 'results', 'MS1_multiple')

Here we generate multiple chemical objects that will be used across samples. The chemical objects are generated by sampling from metabolites in the HMDB database.

In [9]:
# the list of ROI sources created in the previous notebook '01. Download Data.ipynb'
ROI_Sources = [os.path.join(base_dir,'DsDA\\DsDA_Beer\\beer_t10_simulator_files\\')]

# minimum MS1 intensity of chemicals
min_ms1_intensity = 1.75E5

# m/z and RT range of chemicals
rt_range = [(400, 800)]
mz_range = [(100, 400)]

# the number of chemicals in the sample
n_chems = 1000

# maximum MS level (we do not generate fragmentation peaks when this value is 1)
ms_level = 1

# for this experiment, we restrict the sampled chromatograms to be within 20 - 40s in length
# so they are not too big and too small
roi_rt_range = [20, 40]

In [10]:
chems = ChemicalCreator(ps, ROI_Sources, hmdb)
dataset = chems.sample(mz_range, rt_range, min_ms1_intensity, n_chems, ms_level, roi_rt_range=roi_rt_range)
save_obj(dataset, os.path.join(out_dir, 'BaseDataset\\dataset.p'))

Created C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\BaseDataset
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\BaseDataset\dataset.p


In [11]:
for chem in dataset[0:10]:
    print(chem)

KnownChemical - 'C11H12Cl2N2O5' rt=760.27 max_intensity=519800.80
KnownChemical - 'CCl2F2' rt=392.00 max_intensity=1033843.21
KnownChemical - 'C6H12S3' rt=388.96 max_intensity=615608.81
KnownChemical - 'C13H28NO2' rt=413.03 max_intensity=351706.44
KnownChemical - 'C2H7NO2S' rt=411.20 max_intensity=315687.10
KnownChemical - 'C21H36O2' rt=582.23 max_intensity=2117376.67
KnownChemical - 'C8H8O6S' rt=729.16 max_intensity=210368.30
KnownChemical - 'C3Cl6O' rt=686.84 max_intensity=1785002.83
KnownChemical - 'C15H25NO3' rt=457.99 max_intensity=3219728.08
KnownChemical - 'C14H23NO' rt=657.89 max_intensity=340582.10


## Create Multiple Samples

The next section allows us to define classes of biological replicates, each having multiple technical replicates. 

Below we create two biological classes ('class0', 'class1'), each having 10 technical replicates with some noise on the chemical's intensity.

In [12]:
n_samples = [10, 10] # number of files per class
classes = ["class%d" % i for i in range(len(n_samples))] # creates default list of classes
intensity_noise_sd = [1000] # noise on max intensity

In [13]:
classes

['class0', 'class1']

Add intensity changes between different classes

In [14]:
change_probabilities = [0 for i in range(len(n_samples))] # probability of intensity changes between different classes
change_differences_means = [0 for i in range(len(n_samples))] # mean of those intensity changes
change_differences_sds = [0 for i in range(len(n_samples))] # SD of those intensity changes

Add experimental variables (examples in comments)

In [15]:
experimental_classes = None # [["male","female"],["Positive","Negative","Unknown"]]
experimental_probabilitities = None # [[0.5,0.5],[0.33,0.33,0.34]]
experimental_sds = None # [[250],[250]]

Dropout chemicals in different classes

In [16]:
# drop-out chemicals by their probabilities
dropout_probability = 0.2
dropout_probabilities = [dropout_probability for i in range(len(n_samples))]
dropout_numbers = None # drop-out chemicals by an absolute number

# dropout_probabilities = None
# dropout_numbers = 2 # number of chemicals dropped out in each class

Generate multiple samples

In [17]:
save_location = os.path.join(out_dir, 'ChemicalFiles')

In [18]:
multiple_samples = MultiSampleCreator(dataset, n_samples, classes, intensity_noise_sd, 
                                      change_probabilities, change_differences_means, change_differences_sds, dropout_probabilities, dropout_numbers,
                                     experimental_classes, experimental_probabilitities, experimental_sds, save_location=save_location)

Created C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\ChemicalFiles
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\ChemicalFiles\sample_0.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\ChemicalFiles\sample_1.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\ChemicalFiles\sample_2.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\ChemicalFiles\sample_3.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\ChemicalFiles\sample_4.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\ChemicalFiles\sample_5.p
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\ChemicalFiles\sample_6.p
Saving <class 'list'> to C:\Users\joewa\Work

In [19]:
total_samples = np.sum(multiple_samples.n_samples)
total_samples

20

We can also print the chemicals that are missing (removed by drop-out) in each class.

In [20]:
save_obj(multiple_samples.missing_chemicals, os.path.join(out_dir, 'MissingChemicals\\missing_chemicals.p'))
multiple_samples.missing_chemicals

Created C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\MissingChemicals
Saving <class 'list'> to C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\MissingChemicals\missing_chemicals.p


[[KnownChemical - 'C2H7NO2S' rt=411.20 max_intensity=315687.10,
  KnownChemical - 'C21H36O2' rt=582.23 max_intensity=2117376.67,
  KnownChemical - 'C3Cl6O' rt=686.84 max_intensity=1785002.83,
  KnownChemical - 'C16H28O2' rt=429.44 max_intensity=1334366.25,
  KnownChemical - 'C12H26O' rt=543.11 max_intensity=248112.82,
  KnownChemical - 'C7H5F3O' rt=552.54 max_intensity=5263944.27,
  KnownChemical - 'C3H5N3O9' rt=561.92 max_intensity=591758.44,
  KnownChemical - 'C9H19N' rt=652.33 max_intensity=2794399.89,
  KnownChemical - 'C14H11Cl2NO3' rt=616.70 max_intensity=320974.01,
  KnownChemical - 'C9H8ClNO4' rt=402.64 max_intensity=1239959.05,
  KnownChemical - 'C11H16' rt=399.13 max_intensity=619890.30,
  KnownChemical - 'C8H2Cl4O4' rt=428.41 max_intensity=1239937.57,
  KnownChemical - 'C14H25NO' rt=405.78 max_intensity=1372288.84,
  KnownChemical - 'C8H7N3O5' rt=589.49 max_intensity=1431655.82,
  KnownChemical - 'C6H5NO4' rt=425.18 max_intensity=3010609.96,
  KnownChemical - 'C10H16Br2N2O2'

## Run MS1 controller on the samples and generate .mzML files

We can now take the multiple samples created above and generate mass spectral data (.mzML files) using the MS1 controller in ViMMS.

In [ ]:
min_rt = rt_range[0][0]
max_rt = rt_range[0][1]
controllers = defaultdict(list)
controller_to_mzml = {}

mzml_dir = os.path.join(out_dir, 'mzMLFiles')
num_classes = len(n_samples)
sample_idx = 0
for j in range(num_classes): # loop over classes
    num_samples = n_samples[j]
    for i in range(num_samples): # loop over samples for each class
        
        # load the sample
        fname = os.path.join(save_location, 'sample_%d.p' % sample_idx) 
        sample = load_obj(fname)
        sample_idx += 1
        
        # define output .mzML filename
        out_file = 'number_%d_class_%d.mzML' % (i, j)
        out_path = os.path.join(mzml_dir, out_file)
        print('Generating %s' % out_path)

        # run it through the MS1 controller        
        mass_spec = IndependentMassSpectrometer(POSITIVE, sample, density=ps.density_estimator)
        controller = SimpleMs1Controller(mass_spec)
        controller.run(min_rt, max_rt)
        controller.write_mzML('my_analysis', out_path)

        # save the resulting controller
        controllers[j].append(controller)
        controller_to_mzml[controller] = (j, out_file, )

Generating C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\mzMLFiles\number_0_class_0.mzML


401.19485999999904it [00:04, 85.64it/s]                                                                                           


Created C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\mzMLFiles
Generating C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\mzMLFiles\number_1_class_0.mzML


400.8193800000005it [00:04, 97.80it/s]                                                                                            


Generating C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\mzMLFiles\number_2_class_0.mzML


...

401.07606000000055it [00:04, 99.34it/s]                                                                                           


Generating C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\mzMLFiles\number_5_class_1.mzML


400.8506500000002it [00:04, 92.33it/s]                                                                                            


Generating C:\Users\joewa\Work\git\vimms\examples\example_data\results\MS1_multiple\mzMLFiles\number_6_class_1.mzML


  7%|█████▍                                                                       | 27.93370000000067/400 [00:00<00:04, 75.12it/s]

## Print out the missing peaks

The controller object contains all the information about the state of the mass spectrometry process over time. Below we demonstrate this by generating a report of peaks corresponding to a chemical that are present in one class but is missing from the other class. This can be useful in the benchmark evaluation of peak picking or alignment algorithms.

In [ ]:
def get_chem_to_peaks(controller):
    chem_to_peaks = defaultdict(list)
    frag_events = controller.mass_spec.fragmentation_events
    for frag_event in frag_events:
        chem = frag_event.chem
        peaks = frag_event.peaks
        chem_to_peaks[chem].extend(peaks)
    return chem_to_peaks

In [ ]:
for controller, (current_class, mzml_filename) in controller_to_mzml.items():
    controller_peaks = get_chem_to_peaks(controller)
    basename = os.path.basename(mzml_filename)
    front, back = os.path.splitext(mzml_filename)
    outfile = front + '.csv'

    missing_peaks = []            
    for other_class in range(num_classes):
        if current_class == other_class:
            continue

        # get the peaks that are present in current_class but missing in other_class
        missing_chems = multiple_samples.missing_chemicals[other_class]
        for chem in missing_chems:
            peaks = controller_peaks[chem]
            for peak in peaks:
                row = (chem.formula.formula_string, current_class, other_class, peak.mz, peak.rt, peak.intensity)
                missing_peaks.append(row)
    
    # convert to dataframe
    columns = ['formula', 'present_in', 'missing_in', 'mz', 'RT', 'intensity']
    missing_df = pd.DataFrame(missing_peaks, columns=columns)
    missing_df.to_csv(os.path.join(out_dir, 'MissingChemicals', os.path.basename(outfile)))

In [ ]:
missing_df.head(20)